<a href="https://colab.research.google.com/github/Darshan2104/CE_135_Darshan_Tank_ML/blob/main/Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk
import pandas as pd
import re
import string
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import twitter_samples
from sklearn.model_selection import train_test_split
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

tw_data = all_positive_tweets + all_negative_tweets
tw_label = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)),axis=0)

train_x,test_x,train_y,test_y = train_test_split(tw_data,tw_label,test_size=0.30,random_state=13)

In [4]:
class MyTweetLogisticRegressionModel:
  def __transform_data(self,tweets):
    data = []
    for tweet in tweets:
      data.append(self.extract_features(tweet))
    return np.asarray(data)

  def getTransformedData(self,data):
    return self.__transform_data(data)



  def fit(self, x, y, alpha=1e-4, epochs=10,thresval=0.5,record_epoch=False):
    self.thresval = thresval
    self._freq = self.get_freqs(x,y)
    self._shape = 2
    self._inputs = self.__transform_data(x)
    self._tlen = len(y)
    self._bias = tf.Variable(np.random.randn(1), name = "Bias")
    self._weights = tf.Variable(np.random.randn(1,self._shape), name ="Weight")
    self._tinit = tf.compat.v1.global_variables_initializer()
    self._targets = np.asarray(y)
    self.alpha = alpha
    self.epochs = epochs
    self._repoch = record_epoch
    self.__trainModel()
    return
  def __initSaver(self):
    saver = tf.compat.v1.train.Saver()
    self._saver = saver
    self._sesspath = 'TSession'
    return
  def threshold(self,val):
    if val>=self.thresval:
      return 1
    return 0

  def process_tweet(self,tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    tweets_clean = []



    for word in tweet_tokens:
      if word not in stopwords_english and word not in string.punctuation:
        tweets_clean.append(stemmer.stem(word))
    return tweets_clean
  def predict(self, indata):
    with tf.compat.v1.Session() as sess:
      self._saver.restore(sess,save_path=self._sesspath)
      indata = self.__transform_data(indata)
      return sess.run(tf.nn.sigmoid(tf.add(tf.matmul(a=indata,b=self._weights,transpose_b=True),self._bias)))
    print("Failed To Retrieve Session")
    return

  def get_accuracy(self,x,y):
    if len(x)!=len(y):
      print("Dimension are different.")
      return
    return ((x==y).sum())/len(y)

  def get_freqs(self,tweets, ys):
    freqs = dict()
    yslist=np.squeeze(ys).tolist()
    for y, tweet in zip(yslist, tweets):
      for word in self.process_tweet(tweet):
        pair = (word, y)
        if pair not in freqs:
          freqs[pair]=0
        freqs[pair]+=1
    return freqs

  def extract_features(self,tweet):
    freqs = self._freq
    word_l = self.process_tweet(tweet)
    x = np.zeros((1, 2))
    for word in word_l:
      if (word,1) in freqs:
        x[0,0]+= freqs[word,1]
      if (word,0) in freqs:
        x[0,1] += freqs[word,0]
    assert(x.shape == (1, 2))
    return x[0]
  def getAccDets(self):
    if self._repoch:
      return self.__accurary_det
    return


  def __trainModel(self):
    if self._repoch:
      self._err = []
      self._precs = []
      self.__accurary_det = []
    temp0 = tf.nn.sigmoid(tf.add(tf.matmul(a=self._inputs,b=self._weights,transpose_b=True),self._bias))
    err = tf.nn.sigmoid_cross_entropy_with_logits(logits=temp0,labels=self._targets)
    temp = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=self.alpha,name='GradientDescent').minimize(err)
    self.__initSaver()
    with tf.compat.v1.Session() as sess:
      sess.run(self._tinit)
      print("Bias",sess.run(self._bias))
      print("Weights",sess.run(self._weights))
      for epoch in range(self.epochs):
        sess.run(temp)
        __preds = sess.run(temp0)
        acc = ((__preds==self._targets).sum())/self._tlen
        if self._repoch:
          self.__accurary_det.append(acc)
        if epoch % 1000 == 0:
          print("Acc:",acc)
      self._saver.save(sess,self._sesspath)
    return

In [9]:
model = MyTweetLogisticRegressionModel()
model.fit(x=train_x,y=train_y,alpha=1e-4,record_epoch=True,epochs=10000)

Bias [-1.36869362]
Weights [[ 1.56393259 -1.09655332]]
Acc: 0.9347142857142857
Acc: 0.9344285714285714
Acc: 0.9348571428571428
Acc: 0.9354285714285714
Acc: 0.936
Acc: 0.9364285714285714
Acc: 0.9364285714285714
Acc: 0.9364285714285714
Acc: 0.9365714285714286
Acc: 0.9365714285714286
